In [2]:
# ──────────────────────────────────────────────────────────────
# Enhanced GNN Pipeline for First Column Learning
# ──────────────────────────────────────────────────────────────
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import (GCNConv, GATConv, GINConv, global_mean_pool, 
                                global_max_pool, global_add_pool, MessagePassing)
from torch_geometric.data import Data, DataLoader
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d, LayerNorm
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import depolarizing_error, NoiseModel
import pandas as pd
from typing import List, Tuple
from qiskit_ibm_runtime import Sampler, Session, Options

torch.manual_seed(0)
np.random.seed(0)

  
def hellinger_fidelity(p, q):
    """Compute Hellinger fidelity between two probability distributions.

    The Hellinger fidelity is defined as:
        F_H(p, q) = sum_i sqrt(p_i * q_i)

    It ranges from 0 (completely different) to 1 (identical).

    Args:
        p: First probability distribution
        q: Second probability distribution

    Returns:
        Hellinger fidelity (scalar in [0, 1])
    """
    p = np.array(p, dtype=np.float64)
    q = np.array(q, dtype=np.float64)

    # Ensure non-negative
    p = np.maximum(p, 0)
    q = np.maximum(q, 0)

    # Normalize
    if np.sum(p) > 0:
        p = p / np.sum(p)
    if np.sum(q) > 0:
        q = q / np.sum(q)

    return np.sum(np.sqrt(p * q))

# ──────────────────────────────────────────────────────────────
# Original helper functions
# ──────────────────────────────────────────────────────────────
def pauli_noise_model(p1=0.01, p2=0.1):
    nm = NoiseModel()
    one_q_gates = ["h","s","t","x","y","z","rz","sx"]
    two_q_gates = ["cx","cz"]
    one_q_err = depolarizing_error(p1, 1)
    two_q_err = depolarizing_error(p2, 2)
    nm.add_all_qubit_quantum_error(one_q_err, one_q_gates)
    nm.add_all_qubit_quantum_error(two_q_err, two_q_gates)
    return nm

def get_ideal_distribution(circuit, shots=8192):
    """Get ideal (noiseless) distribution from circuit."""
    sim = AerSimulator()
    full_circuit = circuit.copy()
    full_circuit.measure_all()
    
    result = sim.run(full_circuit, shots=shots).result()
    counts = result.get_counts()
    
    n = circuit.num_qubits
    dist = np.zeros(2**n, dtype=np.float32)
    for bitstring, c in counts.items():
        idx = int(bitstring, 2)
        dist[idx] = c / shots
    
    return dist

def random_STHCZ_circuit(n_qubits: int, depth_layers: int, p_cz=0.5):
    qc = QuantumCircuit(n_qubits)
    oneq_gates = ["h","s","t"]
    for _ in range(depth_layers):
        for q in range(n_qubits):
            g = np.random.choice(oneq_gates)
            getattr(qc, g)(q)
        if np.random.rand() < p_cz and n_qubits >= 2:
            a, b = np.random.choice(n_qubits, size=2, replace=False)
            qc.cz(a, b)
        qc.barrier()
    return qc

# ──────────────────────────────────────────────────────────────
# Enhanced Circuit to Graph Conversion
# ──────────────────────────────────────────────────────────────
def circuit_to_graph(circuit: QuantumCircuit, max_depth: int = 20) -> Data:
    """Convert circuit to graph with rich features."""
    n_qubits = circuit.num_qubits
    
    # Extended gate types
    gate_types = ["h", "s", "t", "x", "y", "z", "rz", "sx", "cx", "cz", "barrier", "other"]
    gate_to_idx = {g: i for i, g in enumerate(gate_types)}
    
    node_features = []
    edge_index = []
    edge_attr = []
    
    last_gate_on_qubit = [-1] * n_qubits
    gate_depth = [0] * n_qubits
    
    gate_idx = 0
    total_depth = 0
    
    for inst, qargs, cargs in circuit.data:
        if inst.name == "barrier":
            continue
            
        gate_name = inst.name if inst.name in gate_to_idx else "other"
        
        # Calculate local depth
        involved_qubits = [circuit.qubits.index(q) for q in qargs]
        local_depth = max(gate_depth[q] for q in involved_qubits) + 1
        total_depth = max(total_depth, local_depth)
        
        for q in involved_qubits:
            gate_depth[q] = local_depth
        
        # Node features
        gate_one_hot = [0] * len(gate_types)
        gate_one_hot[gate_to_idx[gate_name]] = 1
        
        qubit_vec = [0] * n_qubits
        for q in qargs:
            qi = circuit.qubits.index(q)
            qubit_vec[qi] = 1
        
        # Additional features
        arity = inst.num_qubits
        is_entangling = 1 if inst.num_qubits > 1 else 0
        is_clifford = 1 if gate_name in ["h", "s", "cx", "cz"] else 0
        is_phase_gate = 1 if gate_name in ["s", "t", "rz"] else 0
        normalized_depth = local_depth / max(max_depth, 1)
        
        # For first column: distance from input matters
        distance_from_input = local_depth / max(total_depth, 1) if total_depth > 0 else 0
        
        node_feat = (gate_one_hot + qubit_vec + 
                    [arity, is_entangling, is_clifford, is_phase_gate, 
                     normalized_depth, distance_from_input])
        node_features.append(node_feat)
        
        # Edges
        for q in qargs:
            qi = circuit.qubits.index(q)
            if last_gate_on_qubit[qi] != -1:
                edge_index.append([last_gate_on_qubit[qi], gate_idx])
                edge_attr.append([1, 0, normalized_depth])  # [sequential, entangling, depth]
                
                edge_index.append([gate_idx, last_gate_on_qubit[qi]])
                edge_attr.append([1, 0, normalized_depth])
            last_gate_on_qubit[qi] = gate_idx
        
        # Entangling edges
        if inst.num_qubits == 2:
            for i, q in enumerate(involved_qubits):
                for j, other_q in enumerate(involved_qubits):
                    if i != j and last_gate_on_qubit[other_q] != -1:
                        edge_index.append([gate_idx, last_gate_on_qubit[other_q]])
                        edge_attr.append([0, 1, normalized_depth])
        
        gate_idx += 1
    
    # Global features
    global_features = [
        n_qubits / 10,
        total_depth / 20,
        gate_idx / 100,
        len([inst for inst, _, _ in circuit.data if inst.num_qubits > 1]) / max(gate_idx, 1)
    ]
    
    # Convert to tensors
    if len(node_features) == 0:
        feature_dim = len(gate_types) + n_qubits + 6
        x = torch.zeros((1, feature_dim), dtype=torch.float32)
        edge_index_tensor = torch.zeros((2, 0), dtype=torch.long)
        edge_attr_tensor = torch.zeros((0, 3), dtype=torch.float32)
    else:
        x = torch.tensor(node_features, dtype=torch.float32)
        if len(edge_index) > 0:
            edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t()
            edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float32)
        else:
            edge_index_tensor = torch.zeros((2, 0), dtype=torch.long)
            edge_attr_tensor = torch.zeros((0, 3), dtype=torch.float32)
    
    global_feat_tensor = torch.tensor(global_features, dtype=torch.float32)
    
    return Data(x=x, edge_index=edge_index_tensor, edge_attr=edge_attr_tensor,
                global_features=global_feat_tensor)


# ──────────────────────────────────────────────────────────────
# Dataset builder
# ──────────────────────────────────────────────────────────────
from dec_matrix_computer_V2 import DECMatrixComputer


def build_graph_dataset(dec_computer, n_qubits=3, depth_layers=6, n_samples=1000,
                       shots=8192, transpile_circuit=False, basis_gates=None, twirl=True, backend=None):
    """Build dataset with graph representations and first column targets.
    If transpile_circuit=True, then we use graph representations of the transpiled circuit.
    If twirl=True, then transpile_circuit must also be true.    
    """
    graphs = []
    columns = []
    circuits = []
    ideal_dists = []
    noisy_dists = []

    if backend is None:
        backend = AerSimulator()
    
    for _ in range(n_samples):
        # Generate circuit
        circ_logical = random_STHCZ_circuit(n_qubits, depth_layers)

        if transpile_circuit:
            if basis_gates is None:
                basis_gates = ['cx', 'id', 'rz', 'sx', 'x']  # IBM basis
                
            circ = transpile(
                circ_logical, 
                basis_gates=basis_gates,
                optimization_level=0  # No optimization to preserve structure
            )
        else:
            circ = circ_logical
            
        # Convert to graph
        graph = circuit_to_graph(circ, max_depth=depth_layers)

        # Get noisy distribution
        if twirl: # Sample from twirled circuit
            noisy_dist = dec_computer.get_full_twirled_distribution(
                    circ_logical,  # Pass the 1-qubit logical circuit
                    num_gate_randomizations=20, # 20 gate twirls
                    num_meas_randomizations=20  # 20 meas twirls (20*20*shots_per)
                                                # This part is a bit tricky.
                                                # A simpler way is to redefine total_shots
                    )
        else:
            noisy_dist = dec_computer.sample_circuit(circ)  # Sample circuit from dec_computer

        # Get first column (for NEC)
        first_col = dec_computer.compute_first_column_fast(circ)
        
        # Get ideal distribution
        ideal_dist = get_ideal_distribution(circ, shots)
        
        # Store
        graph.y = torch.tensor(first_col, dtype=torch.float32).unsqueeze(0)
        graphs.append(graph)
        columns.append(first_col)
        circuits.append(circ)
        ideal_dists.append(ideal_dist)
        noisy_dists.append(noisy_dist)
    
    return graphs, columns, circuits, ideal_dists, noisy_dists

# ──────────────────────────────────────────────────────────────
# Training function
# ──────────────────────────────────────────────────────────────
def train_column_gnn(graphs, n_qubits, n_train=800, epochs=200, batch_size=32,
                    lr=1e-3, model_type="GNN", loss_type="mse",
                    weight_decay=1e-4, scheduler_patience=10):
    """Train GNN for column prediction."""
    
    from graph_networks import ColumnGNN, ColumnGAT, ColumnGIN
    from losses import ColumnL2Loss, StochasticColumnLoss, KLDivergenceLoss
    
    # Split data
    train_graphs = graphs[:n_train]
    val_graphs = graphs[n_train:]
    
    # Data loaders
    train_loader = DataLoader(train_graphs, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_graphs, batch_size=batch_size, shuffle=False)
    
    # Model dimensions
    node_feat_dim = train_graphs[0].x.shape[1]
    output_dim = 2 ** n_qubits
    hidden_dim = 128
    global_feat_dim = train_graphs[0].global_features.shape[0]
    
    # Initialize model
    if model_type == "GNN":
        model = ColumnGNN(node_feat_dim, hidden_dim, output_dim, global_feat_dim)
    elif model_type == "GAT":
        model = ColumnGAT(node_feat_dim, hidden_dim, output_dim, global_feat_dim)
    elif model_type == "GIN":
        model = ColumnGIN(node_feat_dim, hidden_dim, output_dim, global_feat_dim)
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    # Loss function
    if loss_type == "l2":
        loss_fn = ColumnL2Loss()
    elif loss_type == "stochastic":
        loss_fn = StochasticColumnLoss()
    elif loss_type == "kl":
        loss_fn = KLDivergenceLoss()
    else:
        loss_fn = nn.MSELoss()
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', patience=scheduler_patience, factor=0.5
    )
    
    best_val_loss = float('inf')
    best_state = None
    patience_counter = 0
    
    train_losses = []
    val_losses = []
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            
            batch_size_actual = batch.num_graphs
            global_feats = batch.global_features.view(batch_size_actual, -1)
            
            out = model(batch.x, batch.edge_index, batch.batch, global_feats)
            target = batch.y.view(-1, output_dim)
            
            loss = loss_fn(out, target)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item() * batch.num_graphs
        train_loss /= len(train_graphs)
        train_losses.append(train_loss)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch_size_actual = batch.num_graphs
                global_feats = batch.global_features.view(batch_size_actual, -1)
                
                out = model(batch.x, batch.edge_index, batch.batch, global_feats)
                target = batch.y.view(-1, output_dim)
                
                loss = loss_fn(out, target)
                val_loss += loss.item() * batch.num_graphs
        val_loss /= len(val_graphs)
        val_losses.append(val_loss)
        
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch:03d} | Train: {train_loss:.4e} | Val: {val_loss:.4e} | LR: {current_lr:.2e}")
        
        # Early stopping
        if val_loss < best_val_loss - 1e-6:
            best_val_loss = val_loss
            best_state = {k: v.clone() for k, v in model.state_dict().items()}
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= 30:
                print(f"Early stopping at epoch {epoch}")
                break
    
    model.load_state_dict(best_state)
    return model, train_losses, val_losses

# ──────────────────────────────────────────────────────────────
# Evaluation function
# ──────────────────────────────────────────────────────────────
def evaluate_column_model(model, val_graphs, n_qubits):
    """Evaluate column prediction model."""
    model.eval()
    val_loader = DataLoader(val_graphs, batch_size=32, shuffle=False)
    
    all_preds = []
    all_true = []
    output_dim = 2 ** n_qubits
    
    with torch.no_grad():
        for batch in val_loader:
            batch_size_actual = batch.num_graphs
            global_feats = batch.global_features.view(batch_size_actual, -1)
            
            out = model(batch.x, batch.edge_index, batch.batch, global_feats)
            target = batch.y.view(-1, output_dim)
            
            all_preds.append(out.cpu().numpy())
            all_true.append(target.cpu().numpy())
    
    Y_pred = np.concatenate(all_preds, axis=0)
    Y_true = np.concatenate(all_true, axis=0)
    
    # Compute metrics
    metrics = {}
    
    # MSE and MAE
    metrics['mse'] = np.mean((Y_pred - Y_true) ** 2)
    metrics['mae'] = np.mean(np.abs(Y_pred - Y_true))
    
    # L2 norm error
    l2_errors = np.sqrt(np.sum((Y_pred - Y_true) ** 2, axis=1))
    metrics['mean_l2'] = np.mean(l2_errors)
    metrics['std_l2'] = np.std(l2_errors)
    
    # Stochasticity error
    sum_pred = np.sum(Y_pred, axis=1)
    metrics['stochastic_error'] = np.mean(np.abs(sum_pred - 1.0))
    
    # KL divergence (treating as distributions)
    kl_divs = []
    for i in range(len(Y_pred)):
        pred = Y_pred[i] + 1e-10
        true = Y_true[i] + 1e-10
        pred_norm = pred / pred.sum()
        true_norm = true / true.sum()
        kl = np.sum(true_norm * np.log(true_norm / pred_norm))
        kl_divs.append(kl)
    metrics['mean_kl'] = np.mean(kl_divs)
    metrics['std_kl'] = np.std(kl_divs)
    
    # Total variation distance
    tv_distances = 0.5 * np.sum(np.abs(Y_pred - Y_true), axis=1)
    metrics['mean_tv'] = np.mean(tv_distances)
    
    return metrics, Y_pred, Y_true

# ──────────────────────────────────────────────────────────────
# Correction Evaluation Function
# ──────────────────────────────────────────────────────────────
def evaluate_correction(model, val_graphs, val_columns, val_circuits, val_ideal_dists, val_noisy_dists,
                       n_qubits, dec_computer,shots=8192):
    """Evaluate correction performance using both true and predicted columns.
    
    Returns:
        Dictionary with Hellinger fidelities before and after correction
    """
    model.eval()
    
    results = {
        'fidelity_uncorrected': [],
        'fidelity_true_correction': [],
        'fidelity_pred_correction': []
    }
    
    # Get model predictions
    val_loader = DataLoader(val_graphs, batch_size=1, shuffle=False)
    pred_columns = []
    
    with torch.no_grad():
        for batch in val_loader:
            global_feats = batch.global_features.view(1, -1)
            out = model(batch.x, batch.edge_index, batch.batch, global_feats)
            pred_columns.append(out.cpu().numpy()[0])
    
    # Evaluate each circuit
    for i, (circ, true_col, pred_col, ideal_dist, noisy_dist) in enumerate(zip(
            val_circuits, val_columns, pred_columns, val_ideal_dists, val_noisy_dists)):
        
        # Compute uncorrected fidelity
        fid_uncorrected = hellinger_fidelity(noisy_dist, ideal_dist)
        results['fidelity_uncorrected'].append(fid_uncorrected)
        
        # Correct using dec_computer and the correct column
        corrected_true = dec_computer.correct_distribution_properly(noisy_dist, true_col)
        fid_true = hellinger_fidelity(corrected_true, ideal_dist)
        results['fidelity_true_correction'].append(fid_true)

        # Correct using dec_computer and the predicted column
        corrected_pred = dec_computer.correct_distribution_properly(noisy_dist, pred_col)
        fid_pred = hellinger_fidelity(corrected_pred, ideal_dist)
        results['fidelity_pred_correction'].append(fid_pred)

    # Compute statistics
    summary = {
        'mean_fidelity_uncorrected': np.mean(results['fidelity_uncorrected']),
        'std_fidelity_uncorrected': np.std(results['fidelity_uncorrected']),
        'mean_fidelity_true_correction': np.mean(results['fidelity_true_correction']),
        'std_fidelity_true_correction': np.std(results['fidelity_true_correction']),
        'mean_fidelity_pred_correction': np.mean(results['fidelity_pred_correction']),
        'std_fidelity_pred_correction': np.std(results['fidelity_pred_correction']),
    }
    
    return results, summary

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeAthensV2
# ──────────────────────────────────────────────────────────────
# Main comparison script
# ──────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # Settings
    n_qubits = 3
    depth = 3
    n_samples = 2000
    shots = 2048
    
    print("=" * 100)
    print("GNN PIPELINE FOR FIRST COLUMN PREDICTION ON TRANSPILED CIRCUITS (FAKE ATHENS)")
    print("=" * 100)
    
    # Build dataset
    print("\nBuilding graph dataset...")
    
    # Instantiate the fake backend object
    athens_backend = FakeAthensV2()

    dec_computer = DECMatrixComputer(backend=athens_backend, shots=shots, twirl=True)
    
    graphs, columns, circuits, ideal_dists, noisy_dists = build_graph_dataset(
         dec_computer, n_qubits, depth, n_samples, shots=shots, transpile_circuit=True, twirl=True, backend=athens_backend
    )
    print(f"Dataset built: {len(graphs)} circuits")
    print(f"Node features dim: {graphs[0].x.shape[1]}")
    print(f"Target dimension: {2**n_qubits}")
    
    n_train = int(0.8 * n_samples)
    val_graphs = graphs[n_train:]
    val_columns = columns[n_train:]
    val_circuits = circuits[n_train:]
    val_ideal_dists = ideal_dists[n_train:]
    val_noisy_dists = noisy_dists[n_train:]

    results = {}
    correction_results = {}
    
    # Test different combinations
    experiments = [
        ("GNN", "mse", "GCN + MSE"),
        ("GNN", "stochastic", "GCN + Stochastic Loss"),
        ("GNN", "l2", "GCN + L2 Loss"),
        ("GAT", "stochastic", "GAT + Stochastic Loss"),
        ("GIN", "stochastic", "GIN + Stochastic Loss"),
        ("GNN", "kl", "GCN + KL Divergence")
    ]
    
    for model_type, loss_type, name in experiments:
        print("\n" + "=" * 70)
        print(f"Training {name}...")
        print("=" * 70)
        
        model, train_losses, val_losses = train_column_gnn(
            graphs, n_qubits, n_train=n_train, epochs=2000,
            batch_size=32, lr=1e-3, model_type=model_type, loss_type=loss_type
        )
        
        metrics, preds, true = evaluate_column_model(model, val_graphs, n_qubits)
        results[name] = metrics
        
        # Evaluate correction performance
        print(f"\nEvaluating correction for {name}...")
        corr_res, corr_summary = evaluate_correction(
            model, val_graphs, val_columns, val_circuits, 
            val_ideal_dists, val_noisy_ists, n_qubits, 
            dec_computer, shots
        )
        correction_results[name] = corr_summary

    # Print comprehensive results
    print("\n" + "=" * 70)
    print("COMPREHENSIVE EVALUATION RESULTS")
    print("=" * 70)
    
    # Create comparison table
    print("\n{:<25} {:>12} {:>12} {:>12} {:>12} {:>12}".format(
        "Model + Loss", "MSE", "MAE", "L2 Error", "KL Div", "Stoch Err"
    ))
    print("-" * 85)
    
    for name, metrics in results.items():
        print("{:<25} {:>12.4e} {:>12.4e} {:>12.4e} {:>12.4e} {:>12.4e}".format(
            name,
            metrics['mse'],
            metrics['mae'],
            metrics['mean_l2'],
            metrics['mean_kl'],
            metrics['stochastic_error']
        ))
    
    # Print correction results
    print("\n" + "=" * 70)
    print("CORRECTION PERFORMANCE (Hellinger Fidelity)")
    print("=" * 70)
    
    print("\n{:<25} {:>15} {:>15} {:>15}".format(
        "Model + Loss", "Uncorrected", "True Column", "Pred Column"
    ))
    print("-" * 72)
    
    for name, corr in correction_results.items():
        print("{:<25} {:>15.4f} {:>15.4f} {:>15.4f}".format(
            name,
            corr['mean_fidelity_uncorrected'],
            corr['mean_fidelity_true_correction'],
            corr['mean_fidelity_pred_correction']
        ))

    # Find best model
    best_model_name = min(results.keys(), key=lambda x: results[x]['mean_l2'])
    print(f"\n🏆 Best Model (by L2 error): {best_model_name}")
    print(f"   Mean L2 Error: {results[best_model_name]['mean_l2']:.4e}")
    print(f"   Mean KL Divergence: {results[best_model_name]['mean_kl']:.4e}")
    print(f"   Stochasticity Error: {results[best_model_name]['stochastic_error']:.4e}")
    
    # Best correction performance
    best_corr = correction_results[best_model_name]
    print(f"\n   Correction Performance:")
    print(f"   Uncorrected Fidelity: {best_corr['mean_fidelity_uncorrected']:.4f} ± {best_corr['std_fidelity_uncorrected']:.4f}")
    print(f"   True Column Fidelity: {best_corr['mean_fidelity_true_correction']:.4f} ± {best_corr['std_fidelity_true_correction']:.4f}")
    print(f"   Pred Column Fidelity: {best_corr['mean_fidelity_pred_correction']:.4f} ± {best_corr['std_fidelity_pred_correction']:.4f}")
    
    # Improvement metrics
    uncorr_fid = best_corr['mean_fidelity_uncorrected']
    true_fid = best_corr['mean_fidelity_true_correction']
    pred_fid = best_corr['mean_fidelity_pred_correction']
    
    true_improvement = (true_fid - uncorr_fid) / (1 - uncorr_fid) * 100
    pred_improvement = (pred_fid - uncorr_fid) / (1 - uncorr_fid) * 100
    pred_vs_true = pred_fid / true_fid * 100
    
    print(f"\n   Improvement over uncorrected:")
    print(f"   True column: {true_improvement:.1f}% towards ideal")
    print(f"   Pred column: {pred_improvement:.1f}% towards ideal")
    print(f"   Pred achieves {pred_vs_true:.1f}% of true column performance")
    
    # Additional analysis
    print("\n" + "=" * 70)
    print("DETAILED METRICS")
    print("=" * 70)

    print("\nL2 Error Statistics:")
    for name, metrics in results.items():
        print(f"  {name:<25}: μ={metrics['mean_l2']:.4e}, σ={metrics['std_l2']:.4e}")
    
    print("\nKL Divergence Statistics:")
    for name, metrics in results.items():
        print(f"  {name:<25}: μ={metrics['mean_kl']:.4e}, σ={metrics['std_kl']:.4e}")
    
    print("\nTotal Variation Distance:")
    for name, metrics in results.items():
        print(f"  {name:<25}: {metrics['mean_tv']:.4e}")
    
    # Summary
    print("\n" + "=" * 70)
    print("SUMMARY")
    print("=" * 70)
    print(f"""
The GNN approach demonstrates effective noise correction using FWHT:

1. Column Prediction Performance:
   - Best model: {best_model_name}
   - Mean L2 error: {results[best_model_name]['mean_l2']:.4e}
   - Stochasticity maintained: {results[best_model_name]['stochastic_error']:.4e}

2. Correction Performance:
   - Uncorrected fidelity: {uncorr_fid:.4f}
   - True column correction: {true_fid:.4f} ({true_improvement:.1f}% improvement)
   - Predicted column correction: {pred_fid:.4f} ({pred_improvement:.1f}% improvement)
   - Model achieves {pred_vs_true:.1f}% of oracle performance
""")

GNN PIPELINE FOR FIRST COLUMN PREDICTION ON TRANSPILED CIRCUITS (FAKE ATHENS)

Building graph dataset...


C:\Users\zacha\AppData\Local\Temp\ipykernel_6584\3849484040.py:117: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, cargs in circuit.data:
C:\Users\zacha\AppData\Local\Temp\ipykernel_6584\3849484040.py:181: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  len([inst for inst, _, _ in circuit.data if inst.num_qubits > 1]) / max(gate_idx, 1)
